# Assignment 2

In this assignment your goal is to predict the credit application result (not granted / granted) based on multiple features. You'll have to do everything on your own this time, no hints.

The dataset contains 15 features (named A1 - A15) and one target variable (T). We don't know what the features are, we only know what values they take:

* A1: b, a.
* A2: continuous
* A3: continuous
* A4: u, y, l, t
* A5: g, p, gg
* A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff
* A7: v, h, bb, j, n, z, dd, ff, o
* A8: continuous
* A9: t, f
* A10: t, f
* A11: continuous
* A12: t, f
* A13: g, p, s
* A14: continuous
* A15: continuous
* T: +, -

There are 6 features that are continous, 3 true/false variables and 6 categorical variables that take different values each.

From the basic diagnostic of the data, you can see it's a real mess. There are different data types, missing observations etc.

Your task is to build three different classifiers (and one additional GridSearched) that correctly (as much as possible) predict the credit application decision (this is the target variable 'T'). To achieve this, you'll have to do the following:

1. Clean up the dataset - when you read it now, you'll notice that the data is not correctly parsed. The columns that contain numbers are read as 'object' type columns. This is because the missing values in the data set are marked with ?'s.
Deal with it by correctly recognizing ?'s as missing values and dropping them from the dataset.
2. Encode the features - there are a lot of categorical features with char values. You'll need to use [LabelEncoder and OneHotEncoder](https://medium.com/@contactsunny/label-encoder-vs-one-hot-encoder-in-machine-learning-3fc273365621) for them. Also, see if you need to use OneHotEncoder for all of them - for example by checking the correlations between a LabelEncoded feature and the target variable before and after OneHot Encoding. (OneHot encoding doesn't make any sense if your variable actually represents some incremental, hieraarchical relationship - and we don't know it for our dataset).
3. This brings us to the next point - take a look into the data set: plot the histograms, check the skewness, correlations etc. ([also check this](https://seaborn.pydata.org/generated/seaborn.pairplot.html))
4. Train at least 3 diffeent classifiers using cross-validation. There must be at least one *simple* and one ensemble classifier. Compare those classifiers' precissions, recalls, ROC's etc. **Don't forget to use cross-validation.**
5. For one selected classifier run Grid Search and, once the best parameter combination is found, compare it's performance metrics with those of classifiers from point 4.

Scoring:
1. Clean-up: 1 point
2. Encoding: 1 point
3. Data exploration: 2 points (1 point for corellations, histograms, etc., 1 for extras such as attempts at dimensionality reduction or non-linear correlations)
4. Training the 3 classifiers: 1 point for each (3 points total), 1 point for meaningful comparison.
5. Grid Search: 1 point for setting up and training, 1 point for comparing with previous classifiers.


In [2]:
from pandas import read_csv

column_names = [f"A{i}" for i in range(1, 16)]
column_names.append('T')

data = read_csv('data.csv', names=column_names)

In [3]:
print(data.dtypes)

A1      object
A2      object
A3     float64
A4      object
A5      object
A6      object
A7      object
A8     float64
A9      object
A10     object
A11      int64
A12     object
A13     object
A14     object
A15      int64
T       object
dtype: object


In [4]:
print(data.head(20))

   A1     A2     A3 A4 A5  A6 A7    A8 A9 A10  A11 A12 A13    A14    A15  T
0   b  30.83   0.00  u  g   w  v  1.25  t   t    1   f   g  00202      0  +
1   a  58.67   4.46  u  g   q  h  3.04  t   t    6   f   g  00043    560  +
2   a  24.50   0.50  u  g   q  h  1.50  t   f    0   f   g  00280    824  +
3   b  27.83   1.54  u  g   w  v  3.75  t   t    5   t   g  00100      3  +
4   b  20.17   5.62  u  g   w  v  1.71  t   f    0   f   s  00120      0  +
5   b  32.08   4.00  u  g   m  v  2.50  t   f    0   t   g  00360      0  +
6   b  33.17   1.04  u  g   r  h  6.50  t   f    0   t   g  00164  31285  +
7   a  22.92  11.59  u  g  cc  v  0.04  t   f    0   f   g  00080   1349  +
8   b  54.42   0.50  y  p   k  h  3.96  t   f    0   f   g  00180    314  +
9   b  42.50   4.92  y  p   w  v  3.17  t   f    0   t   g  00052   1442  +
10  b  22.08   0.83  u  g   c  h  2.17  f   f    0   t   g  00128      0  +
11  b  29.92   1.83  u  g   c  h  4.33  t   f    0   f   g  00260    200  +
12  a  38.25

In [5]:
# missing values - why is it 0 everywhere? -> do not have any value is null
print(data.isnull().sum())

A1     0
A2     0
A3     0
A4     0
A5     0
A6     0
A7     0
A8     0
A9     0
A10    0
A11    0
A12    0
A13    0
A14    0
A15    0
T      0
dtype: int64


In [6]:
#1. drop the rows have '?'
clean_data = data[data[column_names] != '?']
clean_data = clean_data.dropna()

In [7]:
#Encode feature
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd 

labelencoder = LabelEncoder()
# #collect all the columns have object type
objList = clean_data.select_dtypes(include="object").columns
# # print(objList)
# # for feature in column:
# #     clean_data[feature] = labelencoder.fit_transform(clean_data[feature])
clean_data_en = clean_data[objList].apply(labelencoder.fit_transform)
# #print(clean_data.info())
print(clean_data_en.head(20))
# for feature in objList:
#     accurancy_data = pd.DataFrame(OneHotEncoder().fit_transform(clean_data[[feature]].toarray()), columns=objList)
# print(accurancy_data.head(20))
encoded_data = pd.get_dummies(clean_data, drop_first=True)
encoded_data.head(30)

    A1   A2  A4  A5  A6  A7  A9  A10  A12  A13  A14  T
0    1  153   1   0  12   7   1    1    0    0   68  0
1    0  321   1   0  10   3   1    1    0    0   11  0
2    0   88   1   0  10   3   1    0    0    0   94  0
3    1  123   1   0  12   7   1    1    1    0   31  0
4    1   42   1   0  12   7   1    0    0    2   37  0
5    1  165   1   0   9   7   1    0    1    0  113  0
6    1  176   1   0  11   3   1    0    1    0   54  0
7    0   73   1   0   2   7   1    0    0    0   23  0
8    1  303   2   2   8   3   1    0    0    0   62  0
9    1  249   2   2  12   7   1    0    1    0   15  0
10   1   63   1   0   1   3   0    0    1    0   39  0
11   1  142   1   0   1   3   1    0    0    0   89  0
12   0  215   1   0   8   7   1    0    1    0    0  0
13   1  275   1   0   8   7   0    0    0    0    0  0
14   0  263   1   0  10   7   1    1    1    0    0  0
15   1  207   2   2   8   7   1    1    1    0  103  0
16   1  127   1   0   9   7   1    1    1    0  124  0
17   0   7

,A3,A8,A11,A15,A1_b,A2_15.17,A2_15.75,A2_15.83,A2_15.92,A2_16.00,...,A14_00600,A14_00640,A14_00680,A14_00711,A14_00720,A14_00760,A14_00840,A14_00980,A14_02000,T_-
0,0.00,1.25,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.46,3.04,6,560,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.50,1.50,0,824,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1.54,3.75,5,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5.62,1.71,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4.00,2.50,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1.04,6.50,0,31285,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,11.59,0.04,0,1349,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0.50,3.96,0,314,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,4.92,3.17,0,1442,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
pd.set_option('display.precision', 2)
clean_data_en.corr(method='pearson')

,A1,A2,A4,A5,A6,A7,A9,A10,A12,A13,A14,T
A1,1.00,0.07,6.67e-02,6.31e-02,-1.16e-02,5.72e-02,-0.02,-6.52e-02,5.23e-02,0.08,1.09e-01,0.02
A2,0.07,1.00,-8.66e-02,-1.03e-01,4.09e-02,-2.00e-01,0.22,8.59e-02,7.20e-02,-0.02,-2.95e-02,-0.18
A4,0.07,-0.09,1.00e+00,9.81e-01,-4.50e-02,-2.10e-02,-0.12,-1.58e-01,-9.85e-03,-0.04,-1.09e-03,0.18
A5,0.06,-0.10,9.81e-01,1.00e+00,-5.13e-02,-2.60e-02,-0.13,-1.70e-01,1.75e-03,-0.01,1.39e-02,0.17
A6,-0.01,0.04,-4.50e-02,-5.13e-02,1.00e+00,-9.35e-04,0.11,1.12e-01,6.97e-02,0.02,1.08e-01,-0.12
A7,0.06,-0.20,-2.10e-02,-2.60e-02,-9.35e-04,1.00e+00,-0.03,-1.45e-03,-4.23e-03,0.04,-1.96e-02,0.01
A9,-0.02,0.22,-1.16e-01,-1.29e-01,1.05e-01,-3.50e-02,1.00,4.31e-01,8.99e-02,-0.11,-6.48e-02,-0.74
A10,-0.07,0.09,-1.58e-01,-1.70e-01,1.12e-01,-1.45e-03,0.43,1.00e+00,2.02e-02,-0.23,-7.09e-02,-0.45
A12,0.05,0.07,-9.85e-03,1.75e-03,6.97e-02,-4.23e-03,0.09,2.02e-02,1.00e+00,0.02,1.60e-01,-0.05
A13,0.08,-0.02,-3.79e-02,-1.19e-02,1.72e-02,4.25e-02,-0.11,-2.33e-01,1.71e-02,1.00,6.42e-02,0.10
